In [3]:
import os
import requests
from bs4 import BeautifulSoup
import pdfplumber
from docx import Document
import zipfile
import re
import aspose.words as aw

TOOL chuyển đổi

tách pdf theo dòng

In [4]:
def split_pdf_by_lines(input_file, output_folder, lines_per_file):
    os.makedirs(output_folder, exist_ok=True)
    
    with pdfplumber.open(input_file) as pdf:
        full_text = []
        
        for page in pdf.pages:
            full_text.extend(page.extract_text().split('\n'))
    
    total_lines = len(full_text)
    total_files = (total_lines // lines_per_file) + (1 if total_lines % lines_per_file != 0 else 0)

    for i in range(total_files):
        start_line = i * lines_per_file
        end_line = min((i + 1) * lines_per_file, total_lines)
        file_lines = full_text[start_line:end_line]
        
        output_file = os.path.join(output_folder, f"Trang_{i+1}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("\n".join(file_lines))
    
    print(f"Đã tách file {input_file} thành {total_files} phần trong thư mục {output_folder}.")

DOCX TO TEXT

In [5]:
def docx_to_txt(input_file, output_file):
    doc = Document(input_file)
        
    full_text = [para.text for para in doc.paragraphs if para.text.strip() != '']
    
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write("\n".join(full_text))
    
    print(f"Đã chuyển đổi file {input_file} thành {output_file}.")

PDF TO TEXT

In [6]:
def pdf_to_txt(pdf_dir,output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for file_name in os.listdir(pdf_dir):
        if file_name.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, file_name)
            txt_file_name = f'{os.path.splitext(file_name)[0]}.txt'
            txt_path = os.path.join(output_dir, txt_file_name)

            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                with pdfplumber.open(pdf_path) as pdf:
                    for page_num, page in enumerate(pdf.pages):
                        content = page.extract_text()
                        if content:  # Kiểm tra nếu có nội dung
                            txt_file.write(f'Nội dung của {file_name} - Trang {page_num + 1}:\n{content}\n\n')


CHUYEN DOCX DOC SANG TÊXT VA TACH THANH NHIEU FILE TXT

In [7]:
import os
from docx import Document

def split_docx_by_lines(input_file, output_folder, lines_per_file):
    os.makedirs(output_folder, exist_ok=True)
    
    doc = Document(input_file)
    
    full_text = [para.text for para in doc.paragraphs if para.text.strip() != '']
    
    total_lines = len(full_text)
    total_files = (total_lines // lines_per_file) + (1 if total_lines % lines_per_file != 0 else 0)

    for i in range(total_files):
        start_line = i * lines_per_file
        end_line = min((i + 1) * lines_per_file, total_lines)
        file_lines = full_text[start_line:end_line]
        
        output_file = os.path.join(output_folder, f"Trang_{i+1}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("\n".join(file_lines))
    
    print(f"Đã tách file {input_file} thành {total_files} phần trong thư mục {output_folder}.")

TÁCH DOC NHIÈU FILE TXT THEO DÒNG

In [8]:
import os
import pythoncom
from win32com.client import Dispatch

def split_doc_by_lines(input_file, output_folder, lines_per_file):
    os.makedirs(output_folder, exist_ok=True)
    
    pythoncom.CoInitialize()
    word = Dispatch("Word.Application")
    doc = word.Documents.Open(input_file)
    
    full_text = []
    for para in doc.Paragraphs:
        text = para.Range.Text.strip()
        if text:
            full_text.append(text)
    
    doc.Close()
    word.Quit()
    
    total_lines = len(full_text)
    total_files = (total_lines // lines_per_file) + (1 if total_lines % lines_per_file != 0 else 0)

    for i in range(total_files):
        start_line = i * lines_per_file
        end_line = min((i + 1) * lines_per_file, total_lines)
        file_lines = full_text[start_line:end_line]
        
        output_file = os.path.join(output_folder, f"Trang_{i+1}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("\n".join(file_lines))
    
    print(f"Đã tách file {input_file} thành {total_files} phần trong thư mục {output_folder}.")

TÁCH TXT THEO DÒNG

In [9]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    result = chardet.detect(raw_data)
    return result['encoding']

def split_txt_by_lines(input_file, output_folder, lines_per_file):
    encoding = detect_encoding(input_file)

    os.makedirs(output_folder, exist_ok=True)
    
    with open(input_file, 'r', encoding=encoding) as file:
        lines = file.readlines()
    
    total_lines = len(lines)
    total_files = (total_lines // lines_per_file) + (1 if total_lines % lines_per_file != 0 else 0)

    # Tạo các file theo số lượng dòng
    for i in range(total_files):
        start_line = i * lines_per_file
        end_line = min((i + 1) * lines_per_file, total_lines)
        file_lines = lines[start_line:end_line]
        
        # Tạo tên file cho từng phần
        output_file = os.path.join(output_folder, f"Trang_{i+1}.txt")
        
        # Ghi nội dung vào file
        with open(output_file, 'w', encoding='utf-8') as file:
            file.writelines(file_lines)
    
    print(f"Đã tách file {input_file} thành {total_files} phần trong thư mục {output_folder}.")

TÁCH TXT THEO TRANG

In [10]:
def split_txt_by_pages(input_file, output_folder):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Đọc toàn bộ nội dung của file gốc
    with open(input_file, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Tìm và tách các trang theo "Trang số X"
    pages = re.split(r'(Trang \d+)', content)
    
    # Kết hợp lại các trang với tiêu đề
    combined_pages = []
    for i in range(1, len(pages), 2):
        combined_pages.append(pages[i] + pages[i+1])
    
    # Ghi từng trang vào file riêng biệt
    for page in combined_pages:
        match = re.search(r'Trang (\d+)', page)
        page_number = match.group(1) if match else str(i+1)
        output_file = os.path.join(output_folder, f"Trang_{page_number}.txt")
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(page)
    
    print(f"Đã tách file {input_file} thành {len(combined_pages)} trang trong thư mục {output_folder}.")


TÁCH DOC THEO TRANG

In [4]:
import os
import pythoncom
from win32com.client import Dispatch

def split_doc_by_lines(input_file, output_folder, lines_per_file):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Khởi tạo COM object để tương tác với Microsoft Word
    pythoncom.CoInitialize()
    word = Dispatch("Word.Application")
    doc = word.Documents.Open(input_file)
    
    # Đọc toàn bộ nội dung của file
    full_text = [para.Range.Text.strip() for para in doc.Paragraphs if para.Range.Text.strip() != '']
    
    # Đóng file và Word application
    doc.Close()
    word.Quit()
    
    # Tính số lượng file cần tạo
    total_lines = len(full_text)
    total_files = (total_lines // lines_per_file) + (1 if total_lines % lines_per_file != 0 else 0)

    # Tạo các file theo số lượng dòng
    for i in range(total_files):
        start_line = i * lines_per_file
        end_line = min((i + 1) * lines_per_file, total_lines)
        file_lines = full_text[start_line:end_line]
        
        # Tạo tên file cho từng phần
        output_file = os.path.join(output_folder, f"Trang_{i+1}.txt")
        
        # Ghi nội dung vào file
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("\n".join(file_lines))
    
    print(f"Đã tách file {input_file} thành {total_files} phần trong thư mục {output_folder}.")



NOI NHIEU VAN BAN LUAT

In [49]:
import os
import pythoncom
import time
from win32com.client import Dispatch

def merge_doc_files(input_folder, output_file):
    # Lấy danh sách tất cả các file .doc trong thư mục
    doc_files = [f for f in os.listdir(input_folder) if f.endswith('.doc')]
    doc_files.sort()  # Sắp xếp danh sách file nếu cần

    # Khởi tạo COM object để tương tác với Microsoft Word
    pythoncom.CoInitialize()
    word = Dispatch("Word.Application")
    word.Visible = False  # Không hiển thị ứng dụng Word

    # Tạo một tài liệu mới
    merged_doc = word.Documents.Add()

    for doc_file in doc_files:
        file_path = os.path.join(input_folder, doc_file)
        temp_doc = word.Documents.Open(file_path)
        temp_doc.Content.Copy()
        merged_doc.Range(merged_doc.Content.End - 1).Paste()
        merged_doc.Range(merged_doc.Content.End - 1).InsertBreak(2)  # Thêm ngắt trang giữa các tài liệu
        temp_doc.Close()
        
        # Thêm thời gian chờ nhỏ để đảm bảo các hành động hoàn thành
        time.sleep(0.5)
    
    # Lưu tài liệu hợp nhất
    merged_doc.SaveAs(output_file)
    merged_doc.Close()
    word.Quit()

    print(f"Đã hợp nhất {len(doc_files)} file vào {output_file}")

In [11]:
def clear_licence_in_txt(input_dir):
    # input_dir = 'D:\\Private\\DATA\\hien_phap_txt'
    lines_to_remove = 2  # Số lượng dòng đầu tiên cần xóa

    for file_name in os.listdir(input_dir):
        if file_name.lower().endswith('.txt'):
            file_path = os.path.join(input_dir, file_name)

            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
            
            lines = lines[lines_to_remove:]

            with open(file_path, 'w', encoding='utf-8') as file:
                file.writelines(lines)
        

TÁCH THÀNH TẬP TRAIN VÀ TEST

In [6]:
import random
def copy_file(src_path, dest_path):
    with open(src_path, 'rb') as src_file:
        with open(dest_path, 'wb') as dest_file:
            dest_file.write(src_file.read())

def split_train_test(input_dir, output_dir,title ,train_ratio=0.7):
    os.makedirs(output_dir, exist_ok=True)

    # Tạo các thư mục train và test
    train_dir = os.path.join(output_dir, title +'train')
    test_dir = os.path.join(output_dir, title +'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    txt_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.txt')]
    
    random.shuffle(txt_files)
    
    split_index = int(len(txt_files) * train_ratio)
    
    train_files = txt_files[:split_index]
    test_files = txt_files[split_index:]
    
    def copy_files(files, target_dir):
        for file_name in files:
            src_path = os.path.join(input_dir, file_name)
            dest_path = os.path.join(target_dir, file_name)
            try:
                copy_file(src_path, dest_path)
                print(f"Đã sao chép {src_path} đến {dest_path}")
            except Exception as e:
                print(f"Error copying {src_path}: {e}")

    # Sao chép các tệp vào thư mục train và test
    copy_files(train_files, train_dir)
    copy_files(test_files, test_dir)

In [14]:
def download_files_from_pages1(dir, base_url_template, start_page=1, end_page=5):
    # Đảm bảo thư mục lưu trữ tệp tồn tại
    if not os.path.exists(dir):
        os.makedirs(dir)
    
    for page in range(start_page, end_page + 1):
        # Tạo URL của trang hiện tại
        base_url = base_url_template.format(page=page)
        print(f"Đang xử lý trang: {base_url}")

        # Gửi yêu cầu HTTP và lấy nội dung HTML của trang
        response = requests.get(base_url)
        response.raise_for_status()  # Kiểm tra lỗi HTTP
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Lấy tất cả các liên kết đính kèm
        attachment_links = soup.find_all('a', href=True)
        
        # Trích xuất các liên kết tệp từ mã JavaScript
        file_links = []
        for link in attachment_links:
            href = link['href']
            if 'javascript:downloadfile' in href:
                # Sử dụng regex để trích xuất URL tệp
                match = re.search(r"downloadfile\('([^']+)',\s*'([^']+)'\)", href)
                if match:
                    file_name = match.group(1)
                    file_url = f"https://vbpl.vn{match.group(2)}"
                    # Chỉ thêm các liên kết không phải ZIP
                    if not file_name.lower().endswith('.zip'):
                        file_links.append((file_url, file_name))
        
        print(f"Tìm thấy các liên kết trên trang: {file_links}")

        # Tải xuống các file không phải ZIP
        for file_url, file_name in file_links:
            save_path = os.path.join(dir, file_name)
            
            # In ra đường dẫn để kiểm tra
            print(f"Tải xuống: {file_url}")
            
            if not os.path.exists(save_path):
                try:
                    # Tải file
                    file_response = requests.get(file_url)
                    file_response.raise_for_status()  # Kiểm tra lỗi HTTP
                    with open(save_path, 'wb') as file:
                        file.write(file_response.content)
                    print(f"Đã tải xuống {file_url} và lưu tại {save_path}")
                except requests.RequestException as e:
                    print(f"Lỗi khi tải {file_url}: {e}")

DAN SU

In [114]:
input_file = 'D:\\Private\\DATA\\luat\\dan_su\\dan_su_text\\bo_luat_dan_su.txt'
output_folder = 'D:\\Private\\DATA\\law_data\\dan_su'
split_txt_by_pages(input_file,output_folder)

# input_dir = 'D:\Private\DATA\law_data\dan_su'
# output_dir = 'D:\Private\DATA\law_data_test'
# split_train_test(input_dir, output_dir, train_ratio=0.7)

Đã tách file D:\Private\DATA\luat\dan_su\dan_su_text\bo_luat_dan_su.txt thành 92 trang trong thư mục D:\Private\DATA\law_data\dan_su.


In [119]:

input_dir = 'D:\Private\DATA\law_data\dan_su'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'dan_su', train_ratio)

Đã sao chép D:\Private\DATA\law_data\dan_su\Trang_91.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Trang_91.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Trang_72.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Trang_72.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Trang_32.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Trang_32.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Trang_46.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Trang_46.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Trang_5.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Trang_5.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Trang_34.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Trang_34.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Trang_58.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Trang_58.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Trang_73.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Trang_73.txt
Đã sao chép D:\Private\DATA\law_data\dan_su\Trang_29.txt đến D:\Private\DATA\DATA_LAW\dan_sutrain\Trang_29.txt
Đã 

HINH SU

In [116]:
input_file = 'D:\\Private\\DATA\\luat\\hinh_su\\luat_hinh_su.txt'
output_folder = 'D:\\Private\\DATA\\law_data\\hinh_su'
split_txt_by_pages(input_file,output_folder)



Đã tách file D:\Private\DATA\luat\hinh_su\luat_hinh_su.txt thành 681 trang trong thư mục D:\Private\DATA\law_data\hinh_su.


In [120]:

input_dir = 'D:\Private\DATA\law_data\hinh_su'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'hinh_su ', train_ratio)


Đã sao chép D:\Private\DATA\law_data\hinh_su\Trang_325.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Trang_325.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Trang_659.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Trang_659.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Trang_557.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Trang_557.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Trang_517.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Trang_517.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Trang_448.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Trang_448.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Trang_53.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Trang_53.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Trang_505.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Trang_505.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Trang_621.txt đến D:\Private\DATA\DATA_LAW\hinh_su train\Trang_621.txt
Đã sao chép D:\Private\DATA\law_data\hinh_su\Trang_398.txt đến D:\Private\

LAO DONG

In [117]:
input_file = 'D:\\Private\\DATA\\luat\\lao_dong\\luat_lao_dong.docx'
output_folder = 'D:\\Private\\DATA\\law_data\\lao_dong'
split_docx_by_lines(input_file,output_folder,30)

Đã tách file D:\Private\DATA\luat\lao_dong\luat_lao_dong.docx thành 46 phần trong thư mục D:\Private\DATA\law_data\lao_dong.


In [121]:

input_dir = 'D:\Private\DATA\law_data\lao_dong'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'lao_dong', train_ratio)


Đã sao chép D:\Private\DATA\law_data\lao_dong\Trang_8.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Trang_8.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Trang_27.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Trang_27.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Trang_15.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Trang_15.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Trang_33.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Trang_33.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Trang_39.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Trang_39.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Trang_9.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Trang_9.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Trang_13.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Trang_13.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Trang_5.txt đến D:\Private\DATA\DATA_LAW\lao_dongtrain\Trang_5.txt
Đã sao chép D:\Private\DATA\law_data\lao_dong\Trang_2.txt đến D:\Private\DATA\DATA_LAW

DAT_DAI

In [42]:


# Ví dụ cách sử dụng
input_file = "D:\\Private\\DATA\\luat\\dat_dai\\dat_dai.doc"
output_folder = "D:\\Private\\DATA\\law_data\\dat_dai"
lines_per_file = 10

split_doc_by_lines(input_file, output_folder, lines_per_file)

Đã tách file D:\Private\DATA\luat\dat_dai\dat_dai.doc thành 288 phần trong thư mục D:\Private\DATA\law_data\dat_dai.


In [44]:
input_dir = 'D:\Private\DATA\law_data\dat_dai'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'dat_dai', train_ratio)

Đã sao chép D:\Private\DATA\law_data\dat_dai\Trang_56.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Trang_56.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Trang_211.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Trang_211.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Trang_57.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Trang_57.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Trang_98.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Trang_98.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Trang_278.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Trang_278.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Trang_35.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Trang_35.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Trang_281.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Trang_281.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Trang_226.txt đến D:\Private\DATA\DATA_LAW\dat_daitrain\Trang_226.txt
Đã sao chép D:\Private\DATA\law_data\dat_dai\Trang_140.txt đến D:\Private\DATA\DATA_LAW\

Kinh Te


In [56]:
merge_doc_files('D:\\Private\\DATA\\luat\\kinh_te','D:\\Private\\DATA\\luat\\kinh_te\\kinh_te.doc')

Đã hợp nhất 3 file vào D:\Private\DATA\luat\kinh_te\kinh_te.doc


In [37]:
# Ví dụ cách sử dụng
input_file = "D:\\Private\\DATA\\luat\\kinh_te\\kinh_te.doc"
output_folder = "D:\\Private\\DATA\\law_data\\kinh_te"
lines_per_file = 20

split_doc_by_lines(input_file, output_folder, lines_per_file)

PackageNotFoundError: Package not found at 'D:\Private\DATA\luat\dau_thau\dau_thau.doc'

In [58]:
input_dir = 'D:\Private\DATA\law_data\kinh_te'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'kinh_te', train_ratio)

Đã sao chép D:\Private\DATA\law_data\kinh_te\Trang_126.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Trang_126.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Trang_104.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Trang_104.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Trang_3.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Trang_3.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Trang_152.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Trang_152.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Trang_55.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Trang_55.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Trang_88.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Trang_88.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Trang_74.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Trang_74.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Trang_29.txt đến D:\Private\DATA\DATA_LAW\kinh_tetrain\Trang_29.txt
Đã sao chép D:\Private\DATA\law_data\kinh_te\Trang_160.txt đến D:\Private\DATA\DATA_LAW\kinh

In [26]:
def doc_to_txt(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Lấy tên thư mục gốc để làm tên tệp
    folder_name = os.path.basename(os.path.normpath(input_dir))

    doc_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.doc')]
    
    # Đánh số thứ tự bắt đầu từ 1
    for i, file_name in enumerate(doc_files, start=1):
        doc_path = os.path.join(input_dir, file_name)
        txt_filename = f'{folder_name}_{i}.txt'
        txt_path = os.path.join(output_dir, txt_filename)
        try:
            doc = aw.Document(doc_path)
            doc.save(txt_path)
            print(f"Đã chuyển đổi {doc_path} và lưu tại {txt_path}")
        except Exception as e:
            print(f"Error processing {doc_path}: {e}")


In [27]:
doc_to_txt('D:\Private\DATA\luat\kinh_te','D:\Private\DATA\luat\kinh_te')

Đã chuyển đổi D:\Private\DATA\luat\kinh_te\chung_khoan.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_1.txt
Đã chuyển đổi D:\Private\DATA\luat\kinh_te\kinh_te.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_2.txt
Đã chuyển đổi D:\Private\DATA\luat\kinh_te\Kinh_te_chung_khoan.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_3.txt
Đã chuyển đổi D:\Private\DATA\luat\kinh_te\Kinh_te_doanh_nghiep.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_4.txt
Đã chuyển đổi D:\Private\DATA\luat\kinh_te\KInh_te_thuong_mai.doc và lưu tại D:\Private\DATA\luat\kinh_te\kinh_te_5.txt


In [1]:
import os
import pythoncom
from win32com.client import Dispatch

def split_doc_by_dieu(input_file, output_folder):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Khởi tạo COM object để tương tác với Microsoft Word
    pythoncom.CoInitialize()
    word = Dispatch("Word.Application")
    doc = word.Documents.Open(input_file)
    
    # Đọc toàn bộ nội dung của file
    full_text = [para.Range.Text.strip() for para in doc.Paragraphs if para.Range.Text.strip() != '']
    
    # Đóng file và Word application
    doc.Close()
    word.Quit()
    
    # Tách văn bản thành các đoạn theo từ khóa "Điều"
    dieu_sections = []
    current_section = []
    
    for paragraph in full_text:
        if paragraph.startswith("Điều"):
            if current_section:
                dieu_sections.append(current_section)
                current_section = []
        current_section.append(paragraph)
    if current_section:
        dieu_sections.append(current_section)
    
    # Ghi từng phần vào các file riêng biệt
    for i, section in enumerate(dieu_sections):
        output_file = os.path.join(output_folder, f"Dieu_{i+1}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("\n".join(section))
    
    print(f"Đã tách file {input_file} thành {len(dieu_sections)} phần trong thư mục {output_folder}.")

DAU THAU

In [42]:
# Ví dụ cách sử dụng
input_file = "D:\Private\DATA\luat\dau_thau\dau_thau.doc"
output_folder = "D:\Private\DATA\law_data\dau_thau"
lines_per_file = 20

split_doc_by_lines(input_file, output_folder, lines_per_file)


Đã tách file D:\Private\DATA\luat\dau_thau\dau_thau.doc thành 58 phần trong thư mục D:\Private\DATA\law_data\dau_thau.


In [10]:

input_dir = 'D:\Private\DATA\law_data\dau_thau'
output_dir = 'D:\Private\DATA\DATA_LAW'
train_ratio = 0.7
split_train_test(input_dir, output_dir,'dau_thau', train_ratio)

Đã sao chép D:\Private\DATA\law_data\dau_thau\Trang_26.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Trang_26.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Trang_18.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Trang_18.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Trang_1.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Trang_1.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Trang_38.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Trang_38.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Trang_52.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Trang_52.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Trang_19.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Trang_19.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Trang_7.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Trang_7.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Trang_21.txt đến D:\Private\DATA\DATA_LAW\dau_thautrain\Trang_21.txt
Đã sao chép D:\Private\DATA\law_data\dau_thau\Trang_54.txt đến D:\Private\DATA\DATA_